In [1]:
from google.colab import drive 
from google.colab import files

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install gensim==3.8.3 --quiet

!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install pydot --quiet
!pip install -q sentencepiece
# !pip install -q transformers
# !pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0
!pip install transformers
!pip install pytorch_lightning
# !pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [3]:

import numpy as np
import pandas as pd
from collections import Counter, OrderedDict

import pickle
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text


import sklearn as sk
import os
import nltk
from nltk.corpus import reuters
from nltk.data import find

import matplotlib.pyplot as plt

import re
import yaml

#This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.  
#Make sure your pip install command specifies gensim==3.8.3
import gensim

import numpy as np

from transformers import BertTokenizer, TFBertModel, T5Tokenizer, TFT5ForConditionalGeneration
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
from torchmetrics.text.rouge import ROUGEScore


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pprint import pprint

from pathlib import Path
import logging
import time
from logging.handlers import RotatingFileHandler



import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
import math

# import nltk
# nltk.download('punkt')
# from nltk.tokenize import sent_tokenize
import transformers
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchmetrics
import tensorboard
from tensorboard.backend.event_processing import event_accumulator



# device = 'cuda' if cuda.is_available() else 'cpu'
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers



from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,  BartTokenizer, BartForConditionalGeneration,  BartConfig
)


import gc
import string
import re
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction




In [ ]:
# find index of "chorus" or "hook" and iterate till you hit 1st double new line .

# then iterate through number of counts of chorus and replace it with <extra_id_i>. Get tuple of start and end indices of each chorus.

# This will help us find start and end indices of each verse

 

In [4]:
# then clean strings 
# remove chorus and verse titles

# keep new lines in there. Just remove double new lines
# keep punctuation in there for now

# clean the text
def preprocess(sentence):
    sentence=str(sentence)

    #uncased means words are all lowercased
    sentence = sentence.lower()

    # remove chorus and verse titles
    # remove the brackets and everything in the brackets
    sentence = re.sub("[\[].*?[\]]", "", sentence)
   
    # replace parentheses only
    sentence = re.sub("[\(\)]", "", sentence)

    sentence = sentence.replace("\n", " ")

    # remove double new lines with single new line. So there is a new line space between sections
    # sentence= re.sub("(\n\n)", "\n", sentence)

    return sentence


# p = preprocess(verse_masked_lyrics)
# print(p)

# cleantrain_texts = list(map(preprocess, train_texts))

# #you need to make sure you apply the same preprocessing to the test and validation sets

# ### YOUR CODE HERE

# cleantest_texts = list(map(preprocess, test_texts))
# cleanvalid_texts = list(map(preprocess, valid_texts))
# ### END YOUR CODE

# BART 

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/t5_fine_tuning.ipynb#scrollTo=HS8mNXq6bdxq

https://github.com/Lightning-AI/lightning/discussions/11770

https://shivanandroy.com/fine-tune-t5-transformer-with-pytorch/


In [5]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [6]:
class BartFineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(BartFineTuner, self).__init__()
    # self.hparams = hparams
    self.hparams.update(vars(hparams))
    self.save_hyperparameters()
    
    self.model = BartForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = BartTokenizer.from_pretrained(hparams.tokenizer_name_or_path, model_max_length = hparams.max_seq_length )
    self.acc = torchmetrics.Accuracy(mdmc_average = 'samplewise')
    self.scorer = ROUGEScore() #rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      # doesn't have a max_length method, so it will keep trying to predict the same length as the target ids
      self, input_ids, attention_mask=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        # decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        # takes in labels and shifts right to create decoder input ids shift
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    # print("Output keys: ", outputs.keys())
    # print("Batch: " , batch)
    # print("Batch shape :" , batch.shape)


    # source = batch['source_ids']

    #  Pred Output size :  torch.Size([8, 256, 32128])
    # Target Labels size :  torch.Size([8, 256])


    # values are the probabilities.
    # indices are the indices
    # y_pred = y_max_idx  # predictions are really the inx \in [n_classes] with the highest scores
    
    


  
    
    # print("******")
    # print("Pred Output size : ", preds.size())
    # print("Target Labels size : ", torch.max(labels, torch.tensor([0])).size())


    pred_ids = torch.max(outputs['logits'], 2).indices

    # change pred ids index of blank string token at end to index of PAD token, like target label has
    # go backward
    # DON"T NEED FOR BART
    # for i in range(pred_ids.size(0)):
    #   for j in range(1, pred_ids.size(1) + 1):
    #     # if  value is  3, then replace current valuw ith 0 and break
    #     # print(k[i][j])
    #     if pred_ids[i][-j] == 3:
    #       pred_ids[i][-j]=0
    #     else:
    #       break
  

    # change all the empty strings (word ind = 3) at the end to 0's to represent PAD token ind.
    # for each tensor in pred_ids, maks only 

    pred_str_li = self.tokenizer.batch_decode(pred_ids,skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    # get the values before the cast. set -100 to 1 . Pad token or 1 is the same, it will be ignored in string
    target_ids = torch.max(batch["target_ids"], torch.tensor([1]).cuda())
    # print("target_ids : ", target_ids)
    # print("pred_ids : ", pred_ids)



    # print("Tokenizer max length ", self.tokenizer.model_max_length)

    # print("target_ids arr : ", target_ids.shape)
    # print("pred_ids arr : ", pred_ids.shape)



    target_str_li = self.tokenizer.batch_decode(target_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
 
    
    # predicts what should be in the mask
    
    # get rogue scores for each string pair and avergae it
    
    # rogue score for bathc is average of that batch, loss laredy calculatd that average

    # rouge(preds, target)

    rouge_scores = self.scorer(pred_str_li, target_str_li)
    # torch.mean(torch.Tensor([self.scorer.score(target_str_li[i], pred_str_li[i])['rouge1'] for i in range(len(pred_str_li))]))
    #get f1 score of everything

    rouge1 = rouge_scores['rouge1_fmeasure']
    rouge2 = rouge_scores['rouge2_fmeasure']
    rougeL = rouge_scores['rougeL_fmeasure']
    # rougeLsum = rouge_scores['rougeLsum_fmeasure']

  

    preds_new =  torch.max(pred_ids, torch.tensor([1]).cuda())
    labels_new = torch.max(labels, torch.tensor([1]).cuda())

    # set -100 to 0's
    acc = self.acc(preds_new, labels_new)
    # self.log('training/step_acc', acc)
    # print("Target STR : ", target_str_li)
    # print("Pred   STR : ", pred_str_li)

    # print("Target FULL : ", self.tokenizer.batch_decode(target_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False))
    # print("Pred   FULL : ", self.tokenizer.batch_decode(pred_ids,skip_special_tokens=False, clean_up_tokenization_spaces=False))
    
    # print("LOSS : " +  str(loss.item()) + " , ACC: " + str(acc.item())  +" , ROUGE 1: " + str(rouge1.item())  + " , ROUGE 2: " + str(rouge2.item()) +  " , ROUGE L: " + str(rougeL.item())  )
   

    return loss, acc, rouge1, rouge2, rougeL

  def training_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)

    tensorboard_logs = {"train_loss": loss, "train_acc": acc, "train_rouge1": rouge1, "train_rouge2": rouge2, "train_rougeL": rougeL  }

    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)


    # print("TRAIN STEP: ", tensorboard_logs)

    
    return {"loss": loss, "acc": acc,   "rouge1": rouge1, "rouge2": rouge2,  "rougeL": rougeL, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    avg_train_acc = torch.stack([x["acc"] for x in outputs]).mean()
    avg_train_rouge1 = torch.stack([x["rouge1"] for x in outputs]).mean()
    avg_train_rouge2 = torch.stack([x["rouge2"] for x in outputs]).mean()
    avg_train_rougeL = torch.stack([x["rougeL"] for x in outputs]).mean()

    tensorboard_logs = {"train_loss": avg_train_loss, "train_acc": avg_train_acc, "train_rouge1": avg_train_rouge1, "train_rouge2": avg_train_rouge2, "train_rougeL": avg_train_rougeL }

    self.log('step', self.trainer.current_epoch)
    self.log('avg_train_loss', avg_train_loss)
    self.log('avg_train_acc', avg_train_acc)
    
    self.log('avg_train_rouge1', avg_train_rouge1)
    self.log('avg_train_rouge2', avg_train_rouge2)
    self.log('avg_train_rougeL', avg_train_rougeL)
    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

 
    print("TRAIN Epoch End: ", tensorboard_logs)

    # return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)
    tensorboard_logs = {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    # print("VAL STEP: ", {tensorboard_logs)
    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)

    return {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    
  def validation_epoch_end(self, outputs):
    
    avg_val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
    avg_val_rouge1 = torch.stack([x["val_rouge1"] for x in outputs]).mean()
    avg_val_rouge2 = torch.stack([x["val_rouge2"] for x in outputs]).mean()
    avg_val_rougeL = torch.stack([x["val_rougeL"] for x in outputs]).mean()

    # print("VAL EPOCH END  Keys: ", outputs.keys() )
    
    tensorboard_logs = {"val_loss": avg_val_loss, "val_acc": avg_val_acc, "val_rouge1": avg_val_rouge1, "val_rouge2": avg_val_rouge2, "val_rougeL": avg_val_rougeL}
    # return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    self.log('avg_val_loss', avg_val_loss)
    self.log('avg_val_acc', avg_val_acc)
    self.log('avg_val_rouge1', avg_val_rouge1)
    self.log('avg_val_rouge2', avg_val_rouge2)
    self.log('avg_val_rougeL', avg_val_rougeL)

    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

    print("VAL Epoch End: ", tensorboard_logs)


  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, optimizer_closure=None, 
                    on_tpu=False, using_native_amp=False, 
                    using_lbfgs=False):
    # if on_tpu == True: #self.trainer.use_tpu:
    #   # xm.optimizer_step(optimizer)
    #   optimizer.step()
    # else:
    # optimizer.step(closure=optimizer_closure)
    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams, col = "_b")
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=False, shuffle=False, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams, col = "_b")
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [7]:
logger = logging.getLogger(__name__)



class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      output_val_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_val_results_file, "w") as writer:
        for key in sorted(metrics):
          # if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          #if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

In [8]:
bart_args_dict = dict(
    data_dir="./gdrive/MyDrive/W266 Final Project 2/data/new_data",  #directory for all the input data files # path for data files
    output_dir="./gdrive/MyDrive/W266 Final Project 2/models/BART_base-1/", # path to save the checkpoints
    model_name_or_path='facebook/bart-base',
    tokenizer_name_or_path='facebook/bart-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu= 1,#1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    amp_backend = "apex",
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [ ]:
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', model_max_length = args_dict['max_seq_length'])

In [ ]:
# model will take in only 256 tokens max. During trianing, calling model to get prediciotns will also produce 256 tokens
# only generate function() takes in variable token length to define how many words the model should generate
# tokenizer.model_max_length

In [7]:

class SongDataset(Dataset):

# class ImdbDataset(Dataset):
  def __init__(self, tokenizer, data_dir,  type_path, max_len=256, col = ""):

    self.file_path = os.path.join(data_dir, type_path)
    self.file = self.file_path + ".csv"
    
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []
    self.col = col

    self._build()
  
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    # gets it into 1 dimension
    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
  
  def _build(self):
    self._build_examples_from_files(self.file)

  
  def _build_examples_from_files(self, file):

    # read in 1 sample
    # read in 1 input at a time

    # ierate through df rows

    text_df = pd.read_csv(self.file)
    # print("text_df : ", text_df.shape)

    for index, row  in text_df.iterrows():
      # clean strings here
      #input b for BART
      input = preprocess(row['inputs' + self.col])
      target = preprocess(row['targets' + self.col])
      
       # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [input], max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors="pt"
      )

      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target], max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)



In [ ]:
# dataset = SongDataset(tokenizer, args_dict['data_dir'], 'train',  max_len=256)
# ImdbDataset(tokenizer, 'aclImdb', 'val',  max_len=512)
# len(dataset)

In [ ]:
# data = dataset[15]
# print(tokenizer.decode(data['source_ids']))
# print(tokenizer.decode(data['target_ids']))

## BART Train

In [10]:
# args_dict.update({'data_dir': 'T5', 'output_dir': 'outputs', 'num_train_epochs':2})
bart_args = argparse.Namespace(**bart_args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=bart_args.output_dir, filename='{epoch}-{val_loss:.2f}', monitor="avg_val_loss", mode="min", save_top_k=5
)

# path to save lightning_logs
tb_logger = pl_loggers.TensorBoardLogger(save_dir=bart_args.output_dir)


bart_train_params = dict(
    logger = tb_logger,
    
    accumulate_grad_batches=bart_args.gradient_accumulation_steps,


    gpus=bart_args.n_gpu, 
    # accelerator = "cpu",

    max_epochs=bart_args.num_train_epochs,
    log_every_n_steps = 50, # change based on size of dataset
    # early_stop_callback=False,
    precision= 16 if bart_args.fp_16 else 32,
    amp_level=bart_args.opt_level,
    amp_backend = bart_args.amp_backend,
    gradient_clip_val=bart_args.max_grad_norm,
    enable_checkpointing = True,
    # checkpoint_callback=checkpoint_callback,
    callbacks=[checkpoint_callback, LoggingCallback()],
)



In [5]:
def get_dataset(tokenizer, type_path, args, col):
  return SongDataset(tokenizer=tokenizer, data_dir=args.data_dir,  type_path=type_path, max_len=args.max_seq_length, col = col)

# Fine-tune Model

In [12]:
bart_model = BartFineTuner(bart_args)

In [13]:
# REMOVE CPU here
bart_trainer = pl.Trainer(**bart_train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
# # # clear torch cache

gc.collect()

torch.cuda.empty_cache()



In [15]:
# start fine-tuning
bart_trainer.fit(bart_model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
Missing logger folder: ./gdrive/MyDrive/W266 Final Project 2/models/BART_base-1/lightning_logs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/gdrive/.shortcut-targets-by-id/1tdlBf0ibenbP95uaYV-bH3xJJIgmACVX/W266 Final Project 2/models/BART_base-1 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


VAL Epoch End:  {'val_loss': tensor(3.6333, device='cuda:0'), 'val_acc': tensor(0.1196, device='cuda:0'), 'val_rouge1': tensor(0.3389), 'val_rouge2': tensor(0.1794), 'val_rougeL': tensor(0.2841)}


Training: 0it [00:00, ?it/s]

...

VAL Epoch End:  {'val_loss': tensor(2.5651, device='cuda:0'), 'val_acc': tensor(0.1494, device='cuda:0'), 'val_rouge1': tensor(0.5379), 'val_rouge2': tensor(0.3103), 'val_rougeL': tensor(0.4414)}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:231: UserWarning: You called `self.log('step', ...)` in your `training_epoch_end` but the value needs to be floating point. Converting it to torch.float32.
  f"You called `self.log({self.meta.name!r}, ...)` in your `{self.meta.fx}` but the value needs to"


TRAIN Epoch End:  {'train_loss': tensor(0.1926, device='cuda:0'), 'train_acc': tensor(0.1299, device='cuda:0'), 'train_rouge1': tensor(0.4299), 'train_rouge2': tensor(0.2057), 'train_rougeL': tensor(0.3375)}


Validation: 0it [00:00, ?it/s]

VAL Epoch End:  {'val_loss': tensor(2.4914, device='cuda:0'), 'val_acc': tensor(0.1516, device='cuda:0'), 'val_rouge1': tensor(0.5092), 'val_rouge2': tensor(0.2953), 'val_rougeL': tensor(0.4152)}
TRAIN Epoch End:  {'train_loss': tensor(0.1584, device='cuda:0'), 'train_acc': tensor(0.1474, device='cuda:0'), 'train_rouge1': tensor(0.4553), 'train_rouge2': tensor(0.2435), 'train_rougeL': tensor(0.3628)}


In [ ]:
# 11:10

In [ ]:
bart_args_dict['output_dir']

'./gdrive/MyDrive/W266 Final Project/models/BART_base-K/'

In [16]:
# Save model
# CHANGE THE PATH IF NEEDED
bart_model.model.save_pretrained(bart_args_dict['output_dir'] + 'BART_base-1')
print("Saved model")

Saved model


In [ ]:
# 9:02

In [ ]:
%load_ext tensorboard
%tensorboard --logdir='./gdrive/MyDrive/W266 Final Project/models/BART_base-1/'

<IPython.core.display.Javascript object>

# Bart Base Eval

In [8]:
scorer = ROUGEScore()

accuracy = torchmetrics.Accuracy(mdmc_average = 'samplewise')


def get_test_metrics(model, tokenizer, dataloader, file_path):
  r_log = open(file_path + "rouge.txt", 'w+') #open a text file for logging
  b_log = open(file_path + "bleu.txt", 'w+') #open a text file for logging
  a_log = open(file_path + "accuracy.txt", 'w+')
  rouge1_li = []
  rouge2_li = []
  rougeL_li = []
  bleu_li = []
  accuracy_li = []
  # iterate through each batch  in dataloader
  # for batch_ind in range(len(dataloader)):
  iter_dataloader = iter(dataloader)
  # dataloader is preprocessed
  max_tokens = 100
  i = 0
  for batch in iter_dataloader:
    

    pred_ids = model.generate(input_ids=batch['source_ids'], 
                                attention_mask=batch['source_mask'],  
                                num_beams=4, #9
                                    no_repeat_ngram_size=2,
                                    num_return_sequences=1,
                          #  ,
                                    min_length = max_tokens,
                                    max_length=max_tokens
                                )

    #just get rouge metrics
    # dec = [tokenizer.decode(ids) for ids in outs]

    # get loss and accuracy 
    # list of strings 
    pred_m_ids = pred_ids[:, :max_tokens]
    pred_str = tokenizer.batch_decode(pred_m_ids, skip_special_tokens=True, 
                              clean_up_tokenization_spaces=False)
    
    clean_pred_str = [p.translate(str.maketrans('', '', string.punctuation)).replace("\n", " ") for p in pred_str ]

    # each sentence needs to be in it's own 2 brackets
    clean_pred_str_li = [p.split() for p in clean_pred_str ]

    


    # texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
    # # verses with chorus masked
    # input_str = tokenizer.batch_decode( batch['source_ids'], skip_special_tokens=True, 
    #                           clean_up_tokenization_spaces=False)
    # targets = [tokenizer.decode(ids) for ids in batch['target_ids']]
    target_m_ids =  batch['target_ids'][:, :max_tokens]
    target_str = tokenizer.batch_decode(target_m_ids, skip_special_tokens=True, 
                              clean_up_tokenization_spaces=False)
    
    clean_target_str = [p.translate(str.maketrans('', '', string.punctuation)).replace("\n", " ") for p in target_str ]
    
    clean_target_str_li = [p.split() for p in clean_target_str ]

    # accuracy
    # preds_new =  torch.max(pred_m_ids, torch.tensor([1]).cuda())
    # labels_new = torch.max(target_m_ids, torch.tensor([1]).cuda())
    # 1 accuracy across batch
    acc = accuracy(pred_m_ids, target_m_ids).item()
    accuracy_li.append(acc)


    a_s = "Batch " + str(i) + " = Acc: " +  str(acc)

    a_log.write(a_s + "\n")


    # 1 ROUGE SCORE CLACUALTE ACROSS ENTIRE BATCH
    rouge_scores = scorer(clean_pred_str, clean_target_str)
    rouge1 = rouge_scores['rouge1_fmeasure'].item()
    rouge2 = rouge_scores['rouge2_fmeasure'].item()
    rougeL = rouge_scores['rougeL_fmeasure'].item()
    # print("rougue_scores : ", rouge_scores)
    # print("rouge1 : ", rouge1)
    rouge1_li.append(rouge1)
    rouge2_li.append(rouge2)
    rougeL_li.append(rougeL)
    r_s = "Batch " + str(i) + " = Rouge 1: " + str(rouge1) + " , Rouge 2: "  + str(rouge2) + " , Rouge L: " + str(rougeL)

    # print(r_s)
    r_log.write(r_s + "\n")

    # print("clean_pred_str_li : ")
    print("PRED : ",   clean_pred_str)
    # print("LEN pred: ", len(clean_pred_str_li))
    # print("LEN pred 0: ", len(clean_pred_str_li[0]))
   
    # print("clean_target_str_li : ")
    print("TARGET : ", clean_target_str)
    # print("LEN target : ", len(clean_target_str_li))
    # print("LEN target 0: ", len(clean_target_str_li[0]))
    # 1 score for the entire batch

    batch_bleu = []
    for b_ind in range(len(clean_target_str_li)):
      b = sentence_bleu([clean_target_str_li[b_ind]], clean_pred_str_li[b_ind])
      batch_bleu.append(b)

    bleu = np.average(batch_bleu)
    bleu_li.append(bleu)
    b_s = "Batch " + str(i) + " = Bleu: " + str(bleu)
    # print(b_s)
    b_log.write(b_s + "\n") #write to log

    print("Batch " + str(i) + " = Rouge 1: " + str(rouge1) + " , Rouge 2: "  + str(rouge2) + " , Rouge L: " + str(rougeL) \
              + " , Bleu: " + str(bleu) +  " , Acc: " + str(acc))
  
    i +=1

  avg_rouge1 = np.average(rouge1_li)
  avg_rouge2 = np.average(rouge2_li)
  avg_rougeL = np.average(rougeL_li)
  avg_bleu = np.average(bleu_li)
  avg_acc = np.average(accuracy_li)

  b_log.close()
  r_log.close()
  a_log.close()

  return avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu, avg_acc




In [ ]:

gc.collect()

torch.cuda.empty_cache()

In [12]:

data_args_dict = dict(
    data_dir="./gdrive/MyDrive/W266 Final Project 2/data/new_data/",  #directory for all the input data files # path for data files
    max_seq_length=256,
   
)
data_args = argparse.Namespace(**data_args_dict)


In [ ]:
# Try BART model
bart_base_dir = "./gdrive/MyDrive/W266 Final Project 2/models/BART_base-1/"
bart_base_model = BartForConditionalGeneration.from_pretrained(bart_base_dir + "BART_base-1")
bart_base_tokenizer =  BartTokenizer.from_pretrained('facebook/bart-base', model_max_length = 256)



bart_base_test_dataset = get_dataset(tokenizer=bart_base_tokenizer, type_path="test", args=data_args, col = "_b")
bart_base_dataloader = DataLoader(bart_base_test_dataset, batch_size=8, drop_last=False, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
bart_base_rouge1, bart_base_rouge2, bart_base_rougeL, bart_base_bleu, bart_base_acc = get_test_metrics(bart_base_model, bart_base_tokenizer, bart_base_dataloader, bart_base_dir + "test_logs/")


In [ ]:
print("**** BART Full Test Metrics : ****")
print("Rouge 1 : ", bart_base_rouge1)
print("Rouge 2 : ", bart_base_rouge2)
print("Rouge L : ", bart_base_rougeL)
print("Bleu : ", bart_base_bleu)
print("Acc : ", bart_base_acc)

**** BART Full Test Metrics : ****
Rouge 1 :  0.16246772
Rouge 2 :  0.02335625
Rouge L :  0.10855109
Bleu :  0.00901324356546141
Acc :  0.018910605


# T5 

In [17]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [18]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    # self.hparams = hparams
    self.hparams.update(vars(hparams))
    self.save_hyperparameters()
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, model_max_length = hparams.max_seq_length )
    self.acc = torchmetrics.Accuracy(mdmc_average = 'samplewise')
    self.scorer = ROUGEScore() #rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      # doesn't have a max_length method, so it will keep trying to predict the same length as the target ids
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        # takes in labels and shifts right to create decoder input ids shift
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    # print("Output keys: ", outputs.keys())
    # print("Batch: " , batch)
    # print("Batch shape :" , batch.shape)


    # source = batch['source_ids']

    #  Pred Output size :  torch.Size([8, 256, 32128])
    # Target Labels size :  torch.Size([8, 256])


    # values are the probabilities.
    # indices are the indices
    # y_pred = y_max_idx  # predictions are really the inx \in [n_classes] with the highest scores
    
    


  
    
    # print("******")
    # print("Pred Output size : ", preds.size())
    # print("Target Labels size : ", torch.max(labels, torch.tensor([0])).size())


    pred_ids = torch.max(outputs['logits'], 2).indices

    # change pred ids index of blank string token at end to index of PAD token, like target label has
    # go backward
    for i in range(pred_ids.size(0)):
      for j in range(1, pred_ids.size(1) + 1):
        # if  value is  3, then replace current valuw ith 0 and break
        # print(k[i][j])
        if pred_ids[i][-j] == 3:
          pred_ids[i][-j]=0
        else:
          break
  

    # change all the empty strings (word ind = 3) at the end to 0's to represent PAD token ind.
    # for each tensor in pred_ids, maks only 

    pred_str_li = self.tokenizer.batch_decode(pred_ids,skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    # get the values before the cast. set -100 to 0 . Pad token or 0 is the same, it will be ignored in string
    target_ids = torch.max(batch["target_ids"], torch.tensor([0]).cuda())
    # print("target_ids : ", target_ids.size())
    # print("pred_ids : ", pred_ids.size())
    # print("Tokenizer max length ", self.tokenizer.model_max_length)

    # print("target_ids arr : ", target_ids.shape)
    # print("pred_ids arr : ", pred_ids.shape)



    target_str_li = self.tokenizer.batch_decode(target_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
 
    
    # predicts what should be in the mask
    
    # get rogue scores for each string pair and avergae it
    
    # rogue score for bathc is average of that batch, loss laredy calculatd that average

    # rouge(preds, target)

    rouge_scores = self.scorer(pred_str_li, target_str_li)
    # torch.mean(torch.Tensor([self.scorer.score(target_str_li[i], pred_str_li[i])['rouge1'] for i in range(len(pred_str_li))]))
    #get f1 score of everything

    rouge1 = rouge_scores['rouge1_fmeasure']
    rouge2 = rouge_scores['rouge2_fmeasure']
    rougeL = rouge_scores['rougeL_fmeasure']
    # rougeLsum = rouge_scores['rougeLsum_fmeasure']

  

    preds_new =  torch.max(pred_ids, torch.tensor([0]).cuda())
    labels_new = torch.max(labels, torch.tensor([0]).cuda())

    # set -100 to 0's
    acc = self.acc(preds_new, labels_new)
    # self.log('training/step_acc', acc)
    # print("Target Output : ", target_str_li)
    # print("Pred Output : ", pred_str_li)
    
    # print("LOSS", loss)
    # print("Accuracy", acc)





    return loss, acc, rouge1, rouge2, rougeL

  def training_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)

    tensorboard_logs = {"train_loss": loss, "train_acc": acc, "train_rouge1": rouge1, "train_rouge2": rouge2, "train_rougeL": rougeL  }

    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)


    # print("TRAIN STEP: ", tensorboard_logs)

    
    return {"loss": loss, "acc": acc,   "rouge1": rouge1, "rouge2": rouge2,  "rougeL": rougeL, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    avg_train_acc = torch.stack([x["acc"] for x in outputs]).mean()
    avg_train_rouge1 = torch.stack([x["rouge1"] for x in outputs]).mean()
    avg_train_rouge2 = torch.stack([x["rouge2"] for x in outputs]).mean()
    avg_train_rougeL = torch.stack([x["rougeL"] for x in outputs]).mean()

    tensorboard_logs = {"train_loss": avg_train_loss, "train_acc": avg_train_acc, "train_rouge1": avg_train_rouge1, "train_rouge2": avg_train_rouge2, "train_rougeL": avg_train_rougeL }

    self.log('step', self.trainer.current_epoch)
    self.log('avg_train_loss', avg_train_loss)
    self.log('avg_train_acc', avg_train_acc)
    
    self.log('avg_train_rouge1', avg_train_rouge1)
    self.log('avg_train_rouge2', avg_train_rouge2)
    self.log('avg_train_rougeL', avg_train_rougeL)
    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

 
    print("TRAIN Epoch End: ", tensorboard_logs)

    # return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss, acc, rouge1, rouge2, rougeL = self._step(batch)
    tensorboard_logs = {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    # print("VAL STEP: ", {tensorboard_logs)
    self.log('log', tensorboard_logs,  prog_bar=True, logger=True, on_step=True, on_epoch=True)

    return {"val_loss": loss, "val_acc": acc, "val_rouge1": rouge1, "val_rouge2": rouge2, "val_rougeL": rougeL}

    
  def validation_epoch_end(self, outputs):
    
    avg_val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
    avg_val_rouge1 = torch.stack([x["val_rouge1"] for x in outputs]).mean()
    avg_val_rouge2 = torch.stack([x["val_rouge2"] for x in outputs]).mean()
    avg_val_rougeL = torch.stack([x["val_rougeL"] for x in outputs]).mean()

    # print("VAL EPOCH END  Keys: ", outputs.keys() )
    
    tensorboard_logs = {"val_loss": avg_val_loss, "val_acc": avg_val_acc, "val_rouge1": avg_val_rouge1, "val_rouge2": avg_val_rouge2, "val_rougeL": avg_val_rougeL}
    # return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    self.log('avg_val_loss', avg_val_loss)
    self.log('avg_val_acc', avg_val_acc)
    self.log('avg_val_rouge1', avg_val_rouge1)
    self.log('avg_val_rouge2', avg_val_rouge2)
    self.log('avg_val_rougeL', avg_val_rougeL)

    self.log('log', tensorboard_logs)
    self.log('progress_bar', tensorboard_logs)

    print("VAL Epoch End: ", tensorboard_logs)


  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, optimizer_closure=None, 
                    on_tpu=False, using_native_amp=False, 
                    using_lbfgs=False):
    # if on_tpu == True: #self.trainer.use_tpu:
    #   # xm.optimizer_step(optimizer)
    #   optimizer.step()
    # else:
    # optimizer.step(closure=optimizer_closure)
    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams, col = "")
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=False, shuffle=False, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams, col = "")
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [19]:
logger = logging.getLogger(__name__)



class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      output_val_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_val_results_file, "w") as writer:
        for key in sorted(metrics):
          # if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          #if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))
          writer.write("{} = {}\n".format(key, str(metrics[key])))

In [20]:
t5_args_dict = dict(
    data_dir="./gdrive/MyDrive/W266 Final Project 2/data/new_data",  #directory for all the input data files # path for data files
    output_dir="./gdrive/MyDrive/W266 Final Project 2/models/T5_base-1/", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu= 1,#1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    amp_backend = "apex",
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [21]:
# args_dict.update({'data_dir': 'T5', 'output_dir': 'outputs', 'num_train_epochs':2})
t5_args = argparse.Namespace(**t5_args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=t5_args.output_dir, filename='{epoch}-{val_loss:.2f}', monitor="avg_val_loss", mode="min", save_top_k=5
)

# path to save lightning_logs
tb_logger = pl_loggers.TensorBoardLogger(save_dir=t5_args.output_dir)


t5_train_params = dict(
    logger = tb_logger,
    
    accumulate_grad_batches=t5_args.gradient_accumulation_steps,


    gpus=t5_args.n_gpu, 
    # accelerator = "cpu",

    max_epochs=t5_args.num_train_epochs,
    log_every_n_steps = 50, # change based on size of dataset
    # early_stop_callback=False,
    precision= 16 if t5_args.fp_16 else 32,
    amp_level=t5_args.opt_level,
    amp_backend = t5_args.amp_backend,
    gradient_clip_val=t5_args.max_grad_norm,
    enable_checkpointing = True,
    # checkpoint_callback=checkpoint_callback,
    callbacks=[checkpoint_callback, LoggingCallback()],
)



## Fine-tune T5

In [22]:
t5_model = T5FineTuner(t5_args)

In [23]:
t5_trainer = pl.Trainer(**t5_train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [36]:
# # # clear torch cache

gc.collect()

torch.cuda.empty_cache()

# # #2:14

In [25]:
# start fine-tuning
t5_trainer.fit(t5_model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/gdrive/.shortcut-targets-by-id/1tdlBf0ibenbP95uaYV-bH3xJJIgmACVX/W266 Final Project 2/models/T5_base-1 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


VAL Epoch End:  {'val_loss': tensor(6.0126, device='cuda:0'), 'val_acc': tensor(0.4800, device='cuda:0'), 'val_rouge1': tensor(0.2774), 'val_rouge2': tensor(0.1154), 'val_rougeL': tensor(0.2435)}


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:287: UserWarning: The ``compute`` method of metric _ResultMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  f"The ``compute`` method of metric {self.__class__.__name__}"


VAL Epoch End:  {'val_loss': tensor(2.2634, device='cuda:0'), 'val_acc': tensor(0.5496, device='cuda:0'), 'val_rouge1': tensor(0.5504), 'val_rouge2': tensor(0.3241), 'val_rougeL': tensor(0.4787)}
TRAIN Epoch End:  {'train_loss': tensor(0.1639, device='cuda:0'), 'train_acc': tensor(0.4440, device='cuda:0'), 'train_rouge1': tensor(0.5031), 'train_rouge2': tensor(0.2585), 'train_rougeL': tensor(0.4292)}


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [9]:
t5_model.model.save_pretrained(t5_args_dict['output_dir'] + 'T5_base-1')
print("Saved model")

NameError: ignored

## T5 Eval

In [13]:
# Try T5 model
t5_base_dir = "./gdrive/MyDrive/W266 Final Project 2/models/T5_base-1/"
t5_base_model = T5ForConditionalGeneration.from_pretrained(t5_base_dir + "/T5_base-1")
t5_base_tokenizer =  T5Tokenizer.from_pretrained('t5-base', model_max_length = 256)


t5_base_test_dataset = get_dataset(tokenizer=t5_base_tokenizer, type_path="test", args=data_args, col = "")
t5_base_dataloader = DataLoader(t5_base_test_dataset, batch_size=16, drop_last=False, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:

t5_base_rouge1, t5_base_rouge2, t5_base_rougeL, t5_base_bleu, t5_base_acc = get_test_metrics(t5_base_model, t5_base_tokenizer, t5_base_dataloader, t5_base_dir + "test_logs/")


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

In [ ]:
print("**** T5 Full Test Metrics : ****")
print("Rouge 1 : ", t5_base_rouge1)
print("Rouge 2 : ", t5_base_rouge2)
print("Rouge L : ", t5_base_rougeL)
print("Bleu : ", t5_base_bleu)
print("Acc : ", t5_base_acc)